# Assignment 21 Solutions

##### 1. What is the estimated depth of a Decision Tree trained (unrestricted) on a one million instance training set ?

The estimated depth of a Decision Tree trained on an unrestricted one million instance training set can vary depending on the complexity of the data and the specific parameters used for tree construction. 

##### 2. Is the Gini impurity of a node usually lower or higher than that of its parent? Is it always lower/greater, or is it usually lower/greater ?

The Gini impurity of a node is typically lower than that of its parent. Splitting a node aims to decrease the impurity, resulting in child nodes with lower Gini impurity. It is not always guaranteed that the impurity will be strictly lower due to the nature of the splitting criterion and the distribution of data.

##### 3. Explain if its a good idea to reduce max depth if a Decision Tree is overfitting the training set ?

If a Decision Tree is overfitting the training set, it can be a good idea to reduce the maximum depth of the tree. By limiting the depth, the tree becomes simpler and less prone to capturing noise or specific instances in the training set. This regularization technique helps control overfitting and promotes better generalization to unseen data.

##### 4. Explain if its a  good idea to try scaling the input features if a Decision Tree underfits the training set ?

Scaling the input features is generally not necessary for Decision Trees because they are insensitive to monotonic transformations of the features. Decision Trees make splits based on thresholds without considering the scale of the features. 



##### 5. How much time will it take to train another Decision Tree on a training set of 10 million instances if it takes an hour to train a Decision Tree on a training set with 1 million instances ?

Estimating the training time for a Decision Tree on a larger training set is challenging due to the dependence on factors like hardware, software optimizations, and specific dataset characteristics.

##### 6. Will setting presort=True speed up training if your training set has 100,000 instances ?

Setting presort=True is unlikely to speed up training if the training set has 100,000 instances. The presort option in scikit-learn's Decision Tree implementation is beneficial for smaller datasets, where the pre-sorting step can improve the efficiency of finding the best splits. 

##### 7. Follow these steps to train and fine-tune a Decision Tree for the moons dataset:
1. To build a moons dataset, use make moons(n samples=10000, noise=0.4).
2. Divide the dataset into a training and a test collection with train test split().
3. To find good hyperparameters values for a DecisionTreeClassifier, use grid search with cross-validation (with the GridSearchCV class). Try different values for max leaf nodes.
4. Use these hyperparameters to train the model on the entire training set, and then assess its output on the test set. You can achieve an accuracy of 85 to 87 percent.

In [ ]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Step a: Build the moons dataset
X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

# Step b: Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step c: Fine-tune hyperparameters with grid search and cross-validation
param_grid = {'max_leaf_nodes': [None, 5, 10, 15, 20]}  # Try different values for max leaf nodes
grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters found by grid search
best_params = grid_search.best_params_

# Step d: Train the model with the best hyperparameters on the entire training set
best_model = DecisionTreeClassifier(random_state=42, **best_params)
best_model.fit(X_train, y_train)

# Assess the model's output on the test set
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# Print accuracy
print("Accuracy:", accuracy)


##### 8. Follow these steps to grow a forest:
1. Using the same method as before, create 1,000 subsets of the training set, each containing 100 instances chosen at random. You can do this with Scikit-ShuffleSplit Learn's class.
2. Using the best hyperparameter values found in the previous exercise, train one Decision Tree on each subset. On the test collection, evaluate these 1,000 Decision Trees. These Decision        Trees would likely perform worse than the first Decision Tree, achieving only around 80% accuracy, since they were trained on smaller sets.
3. Now the magic begins. Create 1,000 Decision Tree predictions for each test set case, and keep only the most common prediction (you can do this with SciPy's mode() function). Over the test collection, this method gives you majority-vote predictions.
4. On the test range, evaluate these predictions: you should achieve a slightly higher accuracy than the first model (approx 0.5 to 1.5 percent higher). You've successfully learned a Random Forest classifier!

In [ ]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import mode
from sklearn.metrics import accuracy_score

# Step 1: Create 1,000 subsets of the training set
n_trees = 1000
subset_size = 100

X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

predictions = []

for _ in range(n_trees):
    # Create a random subset of the training set
    indices = np.random.choice(len(X_train), subset_size, replace=False)
    subset_X_train = X_train[indices]
    subset_y_train = y_train[indices]
    
    # Train a Decision Tree on the subset
    tree = DecisionTreeClassifier(random_state=42)
    tree.fit(subset_X_train, subset_y_train)
    
    # Predict on the test set using the trained tree
    tree_predictions = tree.predict(X_test)
    predictions.append(tree_predictions)

# Step 3: Keep the most common prediction for each test set case
ensemble_predictions = mode(predictions, axis=0)[0][0]

# Step 4: Evaluate the predictions on the test set
accuracy = accuracy_score(y_test, ensemble_predictions)

# Print accuracy
print("Accuracy:", accuracy)
